<a href="https://colab.research.google.com/github/DionKimmm/2019_Summer_DL_Prof_Seok/blob/master/190722_Ver2_Deep_Autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Autoencoder

## 필요한 라이브러리 불러오기

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt 
from tensorflow.examples.tutorials.mnist import input_data

## 데이터 불러오기 & 하이퍼파라미터 정의

In [9]:
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

learning_rate = 0.001 
# learning_rate = 0.01
training_epoch = 100
batch_size = 100

# 신경망 레이어 구성 옵션
n_hidden = 256 # 히든 레이어의 뉴런 갯수
n_input = 28*28 # 입력값 크기 - 이미지 픽셀수

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


## 오토인코더 신경망 모델 구성

In [0]:
X = tf.placeholder(tf.float32, [None, n_input])

L1_encoder = tf.layers.dense(inputs=X, units=512, activation=tf.nn.relu) 
L2_encoder = tf.layers.dense(inputs=L1_encoder, units=256, activation=tf.nn.relu)
L3_encoder = tf.layers.dense(inputs=L2_encoder, units=128, activation=tf.nn.relu)

Latent     = tf.layers.dense(inputs=L3_encoder, units=64, activation=tf.nn.relu) 

L3_decoder = tf.layers.dense(inputs=Latent, units=128, activation=tf.nn.relu) 
L2_decoder = tf.layers.dense(inputs=L3_decoder, units=256, activation=tf.nn.relu)
L1_decoder = tf.layers.dense(inputs=L2_decoder, units=512, activation=tf.nn.relu)

Output = tf.layers.dense(inputs=L1_decoder, units=n_input, activation=tf.nn.sigmoid)

## 손실함수 정의 및 최적화 

In [0]:
# 디코더는 인풋과 최대한 같은 결과를 내야 하므로, 디코딩한 결과를 평가하기 # 위해 입력 값인 X 값을 평가를 # 위한 실측 결과 값으로하여 decoder 와의 차이 # 를 손실값으로 설정

cost = tf.reduce_mean(tf.pow(X - Output, 2))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)


## 훈련 데이터를 오토인코더 신경망 모델 학습

In [12]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
total_batch = int(mnist.train.num_examples/batch_size)

for epoch in range(training_epoch):
      total_cost = 0
      for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs})
        total_cost += cost_val
      print('Epoch:', '%04d' % (epoch + 1), 'Avg. cost =', '{:.4f}'.format(total_cost / total_batch))

print('최적화 완료!')


Epoch: 0001 Avg. cost = 0.0365
Epoch: 0002 Avg. cost = 0.0161
Epoch: 0003 Avg. cost = 0.0121
Epoch: 0004 Avg. cost = 0.0104
Epoch: 0005 Avg. cost = 0.0092
Epoch: 0006 Avg. cost = 0.0085
Epoch: 0007 Avg. cost = 0.0079
Epoch: 0008 Avg. cost = 0.0074
Epoch: 0009 Avg. cost = 0.0070
Epoch: 0010 Avg. cost = 0.0068
Epoch: 0011 Avg. cost = 0.0065
Epoch: 0012 Avg. cost = 0.0062
Epoch: 0013 Avg. cost = 0.0060
Epoch: 0014 Avg. cost = 0.0058
Epoch: 0015 Avg. cost = 0.0057
Epoch: 0016 Avg. cost = 0.0055
Epoch: 0017 Avg. cost = 0.0054
Epoch: 0018 Avg. cost = 0.0053


KeyboardInterrupt: ignored

## 합성 결과 확인

In [0]:

# 입력값(위쪽)과 모델이 생성한 값(아래쪽)을 시각적으로 비교
sample_size = 10
samples = sess.run(Output, feed_dict={X: mnist.test.images[:sample_size]})
fig, ax = plt.subplots(2, sample_size, figsize=(sample_size, 2))

for i in range(sample_size):
     ax[0][i].set_axis_off()
     ax[1][i].set_axis_off()
     ax[0][i].imshow(np.reshape(mnist.test.images[i], (28, 28)))
     ax[1][i].imshow(np.reshape(samples[i], (28, 28)))

plt.show()


In [0]:
!nvidia-smi